In [61]:
import numpy as np
from itertools import izip
from math import *
from scipy.optimize import fsolve, root, brentq

In [2]:
R = 8.314 # [kg/(J.K)]

The van der Waals Equation of State is given by
$$
P = \frac{RT}{v - b} - \frac{a}{v^2}
$$

We can write it in a cubic equation form:
$$
v^3 - \left( b + \frac{RT}{P} \right) v^2 + \frac{a}{P}v - \frac{ab}{P} = 0
$$

where we need to find the roots of $v$

In [42]:
class WanDerWaalsEos():
    def __init__(self, a, b, binary_interaction):
        self.a = a
        self.b = b
        self.binary_interaction = binary_interaction
        
    def set_molar_fractions(self, molar_fractions):
        self.molar_fractions = molar_fractions
    
    def update_eos_coefficients(self):
        x = self.molar_fractions
        
        BI = self.binary_interaction

        AIJ = BI * np.sqrt(np.einsum('i,j', self.a, self.a))

        # This variables will be used in the fugacity expression
        self.numerator_coef = np.einsum('ij,j', AIJ, x)
        
        self.mixture_a = np.dot(np.dot(x, AIJ), x)
        self.mixture_b = np.sum(x * self.b)
        
    def calculate_eos_roots(self, pressure, temperature):
        P = pressure
        T = temperature
        p0 = 1.0
        p1 = - (self.mixture_b + (R * T) / P)
        p2 = self.mixture_a / P
        p3 = - self.mixture_a * self.mixture_b / P     
        
        coef_a = (3.0 * p2 - (p1 ** 2)) / 3.0        
        coef_b = (2.0 * (p1 ** 3) - 9.0 * p1 * p2 + 27.0 * p3) / 27.0        
        delta = 0.25 * (coef_b ** 2) + (coef_a ** 3) / 27.0     
                  
        if delta > 0.0:
            # 1 real root, 2 imaginary                 
            const_A =  (-0.5 * coef_b + sqrt(delta)) ** (1.0/3.0)
            const_B =  (-0.5 * coef_b - sqrt(delta)) ** (1.0/3.0)
                  
            single_real_root = const_A + const_B - p1 / 3.0 
            
            return np.array([single_real_root])
        else:
            # 3 real roots
            phi = acos(-0.5 * coef_b / sqrt(-(coef_a ** 3) / 27.0))
            root_1 = 2.0 * sqrt(-coef_a / 3.0) * cos(phi / 3.0) - p1 / 3.0
            root_2 = 2.0 * sqrt(-coef_a / 3.0) * cos(phi / 3.0 + 2.0 * np.pi / 3.0) - p1 / 3.0
            root_3 = 2.0 * sqrt(-coef_a / 3.0) * cos(phi / 3.0 + 4.0 * np.pi / 3.0) - p1 / 3.0

            smallest_root = min(min(root_1,root_2), root_3)
            largest_root = max(max(root_1,root_2), root_3)
                  
            return np.array([smallest_root, largest_root])

    def calculate_fugacity(
        self,
        component_index,
        temperature,
        molar_volume
    ):
        i = component_index
        T = temperature
        v = molar_volume
        x = self.molar_fractions
        x_i = self.molar_fractions[i]
        
        v_minus_b = v - self.mixture_b
        
        BI = self.binary_interaction
        AIJ = BI * np.sqrt(np.einsum('i,j', self.a, self.a))

        ln_f = (  np.log((x_i * R * T) / v_minus_b) 
                + self.b[i] / v_minus_b 
                - self.numerator_coef[i] / (v * R * T))
        
        return np.exp(ln_f) # [Pa]
    
    def calculate_fugacities(
        self,
        pressure,
        temperature,
        molar_volume
    ):
        P = pressure
        T = temperature
        v = molar_volume
        x = self.molar_fractions
        
        v_minus_b = v - self.mixture_b
        
        BI = self.binary_interaction
        AIJ = BI * np.sqrt(np.einsum('i,j', self.a, self.a))

        ln_f = (  np.log((x * R * T) / v_minus_b) 
                + self.b / v_minus_b 
                - self.numerator_coef / (v * R * T))
        
        return (x * P) * np.exp(ln_f) # [Pa]
        

In [135]:
def func_rachford_rice(x, global_molar_fractions, K_values):
    z = global_molar_fractions
    c = 1.0 / (K_values - 1.0)
    return np.sum(z / (c + x))

def deriv_rachford_rice(x, global_molar_fractions, K_values):
    z = global_molar_fractions
    c = 1.0 / (K_values - 1.0)
    return - np.sum(z / ((c + x) ** 2))
    
def calculate_rachford_rice(global_molar_fractions, K_values):
    min_K = np.min(K_values)
    max_K = np.max(K_values)

    min_val = 1.0 / (1.0 - max_K)
    max_val = 1.0 / (1.0 - min_K)

    F_V = root(
        fun=func_rachford_rice,
        x0=0.5,
        args=(global_molar_fractions, K_values),
        jac=deriv_rachford_rice
    )    

    
    return F_V

In [138]:
min_K = np.min(initial_K_values)
max_val = 1.0 / (1.0 - min_K)
print max_val
calculate_rachford_rice(global_molar_fractions, initial_K_values)

1.01096894234


ValueError: object of too small depth for desired array

error: Result from function call is not a proper array of floats.

In [55]:
# Input data
a = np.array([0.55237, 0.42548])
b = np.array([3.0412e-5, 3.738e-5])

delta_ij = 0.47126/sqrt(a[0] * a[1])

binary_interaction = np.array(
    [[1.000000, delta_ij],
     [delta_ij, 1.000000]]
)

molar_fractions = np.array([0.9, 0.1])

# Create EoS object and set properties
eos = WanDerWaalsEos(a, b, binary_interaction)

# Set molar fractions
eos.set_molar_fractions(molar_fractions)

eos.update_eos_coefficients()

volumes = eos.calculate_eos_roots(pressure=100000.0, temperature=300)
print 'Possible roots are %.4e m3/mol and %.4e m3/mol.' % (volumes[0], volumes[1])

f_i = eos.calculate_fugacity(
    component_index=0,
    temperature=300.0,
    molar_volume=volumes[0]
)

print 'Fugacity is %.1f kPa.' % (f_i/1000.0)

f = eos.calculate_fugacities(
    pressure=100000.0,
    temperature=300.0,
    molar_volume=volumes[0]
)

print f/1000.0

Possible roots are 3.7723e-05 m3/mol and 2.4757e-02 m3/mol.
Fugacity is 103639.4 kPa.
[  9.32754424e+09   7.53222824e+08]


In [50]:
def residual_function(x, temperature, pressure, eos, global_molar_fractions):
    T = temperature
    P = pressure
    z = global_molar_fractions
    
    size = x.shape[0]
    
    K = x[0:size-1] # K-values
    K_minus_one = (K - 1.0)
    
    F_V = x[size-1]
    
    print F_V
    if F_V < 0.0:
        F_V = 0.0
    if F_V > 1.0:
        F_V = 1.0
        
    x_L = z / (F_V * K_minus_one + 1.0)
    x_V = K * x_L
    
    # Vapor
    eos.set_molar_fractions(x_V)
    eos.update_eos_coefficients()
    volumes = eos.calculate_eos_roots(P, T)
    
    if volumes.shape[0] > 1:  
        volume_vapor = volumes[1] # Larger value
    else:
        volume_vapor = volumes[0] # Only value

    assert volume_vapor > 0.0, 'Vapor volume < 0.0!'
    
    f_V = eos.calculate_fugacities(P, T, volume_vapor)
    
    # Liquid
    eos.set_molar_fractions(x_L)
    eos.update_eos_coefficients()
    volumes = eos.calculate_eos_roots(P, T)
    
    if volumes.shape[0] > 1:        
        volume_liquid = volumes[0] # Larger value
    else:
        volume_liquid = volumes[0] # Only value

    assert volume_liquid > 0.0, 'Liquid volume < 0.0!'
        
    f_L = eos.calculate_fugacities(P, T, volume_liquid)
    
    residual_fugacity = f_L - f_V 
    residual_mass = np.sum(z * K_minus_one / (1.0 + F_V * K_minus_one))
    residual = np.append(residual_fugacity, residual_mass)

    print f_L, f_V
    return residual

In [113]:
def calculate_initial_K_values(
    pressure,
    temperature,
    critical_pressure,
    critical_temperature,
    acentric_factor
):
    P = pressure
    T = temperature
    P_c = critical_pressure
    T_c = critical_temperature
    omega = acentric_factor
    
    return (P_c / P) * np.exp(5.37 * (1.0 + omega) * (1.0 - (T_c / T)))

def calculate_m_function(acentric_factor):
    return np.where(
        acentric_factor < 0.49, 
        0.37464 + 1.54226 * acentric_factor - 0.26992 * acentric_factor **2, 
        0.3796 + 1.485 * acentric_factor - 0.1644 * acentric_factor **2 + 0.01667 * acentric_factor ** 3
    )

In [106]:
temperature = 366.5
pressure = 150.0 * 1.0e5

critical_pressure = 101325.0 * np.array([45.45, 51.29, 39.89, 31.95, 27.91, 17.71, 12.52]) # [atm]
critical_temperature = np.array([189.2, 305.4, 395.8, 485.1, 592.0, 697.1, 804.4]) # [K]
acentric_factor = np.array([0.00891, 0.11352, 0.17113, 0.26910, 0.34196, 0.51730, 0.72755]) # [-]
molar_mass = 0.001 * np.array([16.38, 31.77, 50.64, 77.78, 118.44, 193.95, 295.30]) # [g/mol]
omega_a = np.array([0.344772, 0.521974, 0.514972, 0.419169, 0.485943, 0.570583, 0.457236]) # [-]
omega_b = np.array([0.063282, 0.099825, 0.107479, 0.093455, 0.07486, 0.101206, 0.077796]) # [-]

binary_interaction = np.array(
[[ 0.000000,  0.000622, -0.002471,  0.011418, -0.028367, -0.100000, 0.206868],
 [ 0.000622,  0.000000, -0.001540,  0.010046,  0.010046,  0.010046, 0.010046],
 [-0.002471, -0.001540,  0.000000,  0.002246,  0.002246,  0.002246, 0.002246],
 [ 0.011418,  0.010046,  0.002246,  0.000000,  0.000000,  0.000000, 0.000000],
 [-0.028367,  0.010046,  0.002246,  0.000000,  0.000000,  0.000000, 0.000000],
 [-0.100000,  0.010046,  0.002246,  0.000000,  0.000000,  0.000000, 0.000000],
 [ 0.206868,  0.010046,  0.002246,  0.000000,  0.000000,  0.000000, 0.000000]]
)

global_molar_fractions = np.array([0.6793, 0.099, 0.1108, 0.045, 0.05011, 0.0134, 0.00239])



In [124]:
# TEST PROBLEM PHASE BEHAVIOUR WHITSON PROBLEM 18 APPENDIX
temperature = (280.0 + 459.67) * 5.0 / 9.0
pressure = 500.0 * 6894.75729

critical_pressure = 6894.75729 * np.array([667.8, 550.7, 304.0]) # [atm]
critical_temperature = (5.0 / 9.0) * np.array([343.0, 765.3, 1111.8]) # [K]
acentric_factor = np.array([0.011500, 0.192800, 0.490200]) # [-]
molar_mass = 0.001 * np.array([16.04, 58.12, 142.29]) # [g/mol]
omega_a = np.array([0.45724, 0.45724, 0.45724]) # [-]
omega_b = np.array([0.07780, 0.07780, 0.07780]) # [-]

binary_interaction = np.array(
[[0.000000,  0.000000, 0.000000],
 [0.000000,  0.000000, 0.000000],
 [0.000000,  0.000000, 0.000000]]
)

global_molar_fractions = np.array([0.5, 0.42, 0.08])

fugacity_expected = np.array([294.397, 148.342, 3.02385]) * 6894.75729
K_values_expected = np.array([6.65071, 0.890061, 0.03624])
x_expected = np.array([0.08588, 0.46349, 0.45064])
y_expected = np.array([0.57114, 0.41253, 0.01633])
print fugacity_expected

[ 2029795.86190413  1022782.08591318    20848.71183137]


In [125]:
m = calculate_m_function(acentric_factor)
alpha = (1.0 + m * (1.0 - np.sqrt(temperature / critical_temperature)))
a = alpha * (omega_a * (R * critical_temperature) ** 2) / critical_pressure
b = (omega_b * R * critical_temperature) / critical_pressure

initial_vapor_fraction = 0.5

initial_K_values = calculate_initial_K_values(
    pressure,
    temperature,
    critical_pressure,
    critical_temperature,
    acentric_factor
)
print initial_K_values
x0 = np.append(initial_K_values, initial_vapor_fraction)

# Create EoS object and set properties
eos = WanDerWaalsEos(a, b, binary_interaction)

F_V = calculate_rachford_rice(global_molar_fractions, initial_K_values)
print F_V
#result = fsolve(
#    func=residual_function,
#    x0=x0,
#    args=(temperature, pressure, eos, global_molar_fractions),
    #full_output=True
#)

#print result

[  2.45895248e+01   8.82173476e-01   1.08499301e-02]
0.5
[  1.86459607e+10   2.54658197e+12   3.60890809e+11] [  1.12741818e+13   1.98182659e+12   4.24844207e+07]
0.5
[  1.86459607e+10   2.54658197e+12   3.60890809e+11] [  1.12741818e+13   1.98182659e+12   4.24844207e+07]
0.5
[  1.86459607e+10   2.54658197e+12   3.60890809e+11] [  1.12741818e+13   1.98182659e+12   4.24844207e+07]
0.5
[  1.86459602e+10   2.54658197e+12   3.60890809e+11] [  1.12741819e+13   1.98182659e+12   4.24844207e+07]
0.5


AssertionError: Liquid volume < 0.0!

[ 2029795.86190413  1022782.08591318    20848.71183137]


In [158]:
bb = np.array([1,2,3,4])

print bb[bb.shape[0]-1]

4


In [201]:
aa = np.array([1,1,1,3,3,3])

mm = np.where(aa < 2, aa + 0.5, aa + 0.8)
print mm

mm = np.where(aa < 2)
mm1 = np.where(aa > 2)

res
print mm[0], mm1[0]

[ 1.5  1.5  1.5  3.8  3.8  3.8]


NameError: name 'res' is not defined

In [199]:
print aa < 2

[ True  True  True False False False]


In [256]:
aa = np.array([1,2])

In [259]:
if aa.shape > 1:
    print 'here'
    
if bb.shape > 1:
    print 'here'